In [34]:
import pandas as pd
import numpy as np

df = pd.read_excel(r"C:\Users\lawsonsa\Downloads\01-25-2022 Picklist of 44 miRNAs (modified).xlsx", sheet_name = 'Sort by plate', index_col='Sample Group')

replicates = 6
gradient = [150, 50, 15, 5, 0] # in nanoliters

screen_hits = ['hsa-miR-137 + hsa-miR-34b-5p',
 'hsa-miR-3714 + hsa-miR-450b-3p',
 'hsa-miR-128-3p + hsa-miR-3714',
 'hsa-miR-10b-5p + hsa-miR-137',
 'hsa-miR-3714 + hsa-miR-3937',
 'hsa-miR-137 + hsa-miR-449a']

rows = ['B','C','D','E','F','G','H','I','J','K','L','M','N','O']
cols = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
wells = [row+str(col) for row in rows for col in cols]

In [39]:
allmirs = df.index

# make sure names of screen_hits match to the source, mir-137 is often troublesome
j = False
for h in screen_hits:
    mir1, mir2 = h.split(' + ')
    
    if mir1 not in allmirs:
        print(f'{mir1} unrecognized')
        j = True
    if mir2 not in allmirs:
        print(f'{mir2} unrecognized')
        j = True
if not j:
    print('All sources for necessary miRs have been located')

All sources for necessary miRs have been located


In [25]:
def pairwise(lst):
    """
    Generate a list of tuples.
    """
    m=[]
    for i in lst:
        for j in lst:
            m.append((i, j))
    return m
print('Doses per combination per replicate:')
pairwise(gradient)

Doses per combination per replicate:


[(150, 150),
 (150, 50),
 (150, 15),
 (150, 5),
 (150, 0),
 (50, 150),
 (50, 50),
 (50, 15),
 (50, 5),
 (50, 0),
 (15, 150),
 (15, 50),
 (15, 15),
 (15, 5),
 (15, 0),
 (5, 150),
 (5, 50),
 (5, 15),
 (5, 5),
 (5, 0),
 (0, 150),
 (0, 50),
 (0, 15),
 (0, 5),
 (0, 0)]

In [40]:
import math

doses = pairwise(gradient) * replicates
print(f'{len(doses)} total wells per combination (all replicates combined)')

available_wells_in_plate = len(wells) #num wells available per plate
total_wells_needed = len(screen_hits) * len(doses) #num wells needed
total_plates_needed = int(math.ceil(total_wells_needed / available_wells_in_plate)) #num plates needed
spare_wells = (available_wells_in_plate*total_plates_needed)-total_wells_needed
print(f'Num wells per 384-plate available: {available_wells_in_plate}\nNum wells needed for screen: {total_wells_needed}\nNum plates needed for screen: {total_plates_needed}\nSpare wells on last plate, not including deliberately excluded edges: {spare_wells}')

150 total wells per combination (all replicates combined)
Num wells per 384-plate available: 252
Num wells needed for screen: 900
Num plates needed for screen: 4
Spare wells on last plate, not including deliberately excluded edges: 108


In [28]:
dose_drug_combo_dict = {}

for i in range(0, len(screen_hits)):
    for j in range(0, len(doses)):
        dose_drug_combo_dict[str(j) + screen_hits[i]] = [doses[j][0], doses[j][1], screen_hits[i].split(' + ')[0], screen_hits[i].split(' + ')[1]]

print(f'Number of drug dose combinations generated: {len(dose_drug_combo_dict.values())}\n\nThe first 10 wells look like this:')
print(list(dose_drug_combo_dict.values())[:10])

Number of drug dose combinations generated: 900

The first 10 wells look like this:
[[150, 150, 'hsa-miR-137', 'hsa-miR-34b-5p'], [150, 50, 'hsa-miR-137', 'hsa-miR-34b-5p'], [150, 15, 'hsa-miR-137', 'hsa-miR-34b-5p'], [150, 5, 'hsa-miR-137', 'hsa-miR-34b-5p'], [150, 0, 'hsa-miR-137', 'hsa-miR-34b-5p'], [50, 150, 'hsa-miR-137', 'hsa-miR-34b-5p'], [50, 50, 'hsa-miR-137', 'hsa-miR-34b-5p'], [50, 15, 'hsa-miR-137', 'hsa-miR-34b-5p'], [50, 5, 'hsa-miR-137', 'hsa-miR-34b-5p'], [50, 0, 'hsa-miR-137', 'hsa-miR-34b-5p']]


In [29]:
needed_wells = list(dose_drug_combo_dict.values())

target = pd.DataFrame(columns=['Source Plate Barcode ', 'Sample ID', 'Source Plate Name',
       'Source well', 'Row', 'Col', 'Sample Name', 'Sample Group',
       'Sample Comment', 'Destination well', 'Destination Plate Barcode',
       'Destination Volume'])

for plate in range(total_plates_needed):
    for well in wells:
        ## iterate through all wells of all plates, and load up the well with combos and doses. 
        if len(needed_wells) != 0:
            conc1, conc2, mir1, mir2 = needed_wells[0]
            target = target.append(pd.concat([df.loc[mir1][:7], pd.Series({'Sample Group' : mir1, 'Sample Comment': mir1 + ' + ' + mir2, 'Destination well':well, 'Destination Plate Barcode':plate+1, 'Destination Volume':conc1})]), ignore_index=True)
            target = target.append(pd.concat([df.loc[mir2][:7], pd.Series({'Sample Group' : mir2, 'Sample Comment': mir1 + ' + ' + mir2, 'Destination well':well, 'Destination Plate Barcode':plate+1, 'Destination Volume':conc2})]), ignore_index=True)
            needed_wells.pop(0)

In [30]:
target.to_csv('pairwise_echo_instructions.csv')
target

,Source Plate Barcode,Sample ID,Source Plate Name,Source well,Row,Col,Sample Name,Sample Group,Sample Comment,Destination well,Destination Plate Barcode,Destination Volume
0,CPE200F7,ASO21XWC,Hm Mimic v21-1,D9,D,9,MIMAT0000429,hsa-miR-137,hsa-miR-137 + hsa-miR-34b-5p,B4,1,150
1,CPE200F7,ASO21XXI,Hm Mimic v21-1,L5,L,5,MIMAT0000685,hsa-miR-34b-5p,hsa-miR-137 + hsa-miR-34b-5p,B4,1,150
2,CPE200F7,ASO21XWC,Hm Mimic v21-1,D9,D,9,MIMAT0000429,hsa-miR-137,hsa-miR-137 + hsa-miR-34b-5p,B5,1,150
3,CPE200F7,ASO21XXI,Hm Mimic v21-1,L5,L,5,MIMAT0000685,hsa-miR-34b-5p,hsa-miR-137 + hsa-miR-34b-5p,B5,1,50
4,CPE200F7,ASO21XWC,Hm Mimic v21-1,D9,D,9,MIMAT0000429,hsa-miR-137,hsa-miR-137 + hsa-miR-34b-5p,B6,1,150
...,...,...,...,...,...,...,...,...,...,...,...,...
1795,CPE200F7,ASO21XZD,Hm Mimic v21-1,G8,G,8,MIMAT0001541,hsa-miR-449a,hsa-miR-137 + hsa-miR-449a,I19,4,15
1796,CPE200F7,ASO21XWC,Hm Mimic v21-1,D9,D,9,MIMAT0000429,hsa-miR-137,hsa-miR-137 + hsa-miR-449a,I20,4,0
1797,CPE200F7,ASO21XZD,Hm Mimic v21-1,G8,G,8,MIMAT0001541,hsa-miR-449a,hsa-miR-137 + hsa-miR-449a,I20,4,5
1798,CPE200F7,ASO21XWC,Hm Mimic v21-1,D9,D,9,MIMAT0000429,hsa-miR-137,hsa-miR-137 + hsa-miR-449a,I21,4,0


In [31]:
def make_plate_df(plate_name):
    plate_df = pd.DataFrame(index=rows, columns=cols)
    for well in wells:
        try:
            drugs = target[(target['Destination well']==well) & (target['Destination Plate Barcode']==plate_name)].reset_index()
            mir1 = 
            mir2 = 
            dose1 = str(drugs['Destination Volume'][0])
            dose2 = str(drugs['Destination Volume'][1])
            plate_df.loc[well[0],int(well[1:])] = drugs['Sample Group'].values[0] +f'({dose1}nl)+' + drugs['Sample Group'].values[1] +f'({dose2}nl)'

        except IndexError:
            plate_df.loc[well[0],int(well[1:])] = 'Untreated'
        except KeyError:
            plate_df.loc[well[0],int(well[1:])] = 'Untreated'
    return plate_df

plate_dfs = {}
for plate in target['Destination Plate Barcode'].unique():
    plate_dfs[plate] = make_plate_df(plate)

In [41]:
def make_plate_df(plate_name):
    plate_df = pd.DataFrame(index=rows, columns=cols)
    for well in wells:
        try:
            drugs = target[(target['Destination well']==well) & (target['Destination Plate Barcode']==plate_name)].reset_index()
            mir1 = drugs['Sample Group'].values[0]
            mir2 = drugs['Sample Group'].values[1]
            dose1 = str(drugs['Destination Volume'][0])
            dose2 = str(drugs['Destination Volume'][1])
            plate_df.loc[well[0],int(well[1:])] =  mir1 +f'({dose1}nl)+' + mir2 +f'({dose2}nl)'

        except IndexError:
            plate_df.loc[well[0],int(well[1:])] = 'Untreated'
        except KeyError:
            plate_df.loc[well[0],int(well[1:])] = 'Untreated'
    return plate_df

plate_dfs = {}
for plate in target['Destination Plate Barcode'].unique():
    plate_dfs[plate] = make_plate_df(plate)

In [32]:
for plate in plate_dfs.keys():
    plate_dfs[plate].to_csv(f'plate {plate}.csv')